## Import Packages

In [3]:
import sys
sys.path.append('../../')

In [9]:
%load_ext autoreload
# import gdown
# import time
import h5py
# import pyUSID as usid
import numpy as np

# import matplotlib.pyplot as plt
# from matplotlib.patches import ConnectionPatch


from src.m3_learning.be.util import print_be_tree, SHO_Fitter
# from src.m3_learning.nn.random import random_seed
# from src.m3_learning.viz.style import style
# from src.m3_learning.be.processing import convert_real_imag
# from src.m3_learning.viz.layout import layout_fig
# from src.m3_learning.util.h5_util import make_dataset, make_group
from scipy.signal import resample
# from scipy import fftpack



# style("default")
# random_seed(seed=42)

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# Sets path to file
path = r"C:/Users/jca92/Documents/codes/m3_learning/m3_learning/papers/2023_Rapid_Fitting/data/"

In [5]:
print_be_tree(path + "data_file.h5")

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Raw_Data-SHO_Fit_000
      --------------------
      ├ Fit
      ├ Guess
      ├ Spectroscopic_Indices
      ├ Spectroscopic_Values
      ├ completed_fit_positions
      ├ completed_guess_positions
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS

### Extract Data

In [21]:
# Opens the data file
h5_f = h5py.File(path + "data_file.h5", "r+")

# number of samples per SHO fit
num_bins = h5_f["Measurement_000"].attrs["num_bins"]

# number of pixels in the image
num_pix = h5_f["Measurement_000"].attrs["num_pix"]

# number of pixels in x and y dimensions
num_pix_1d = int(np.sqrt(num_pix))

# number of DC voltage steps
voltage_steps = h5_f["Measurement_000"].attrs["num_udvs_steps"]

# # sampling rate
# sampling_rate = h5_f["Measurement_000"].attrs["IO_rate_[Hz]"]

# # BE bandwidth
# be_bandwidth = h5_f["Measurement_000"].attrs["BE_band_width_[Hz]"]

# # BE center frequency
# be_center_frequency = h5_f["Measurement_000"].attrs["BE_center_frequency_[Hz]"]

# Frequency Vector in Hz
frequency_bin = h5_f["Measurement_000"]["Channel_000"]["Bin_Frequencies"][:]

# Resampled frequency vector
wvec_freq = resample(frequency_bin, 80)

# extracting spectroscopic values
spectroscopic_values = h5_f["Measurement_000"]["Channel_000"]["Spectroscopic_Values"]

# # extract the real and imaginary components
# real = h5_f["Measurement_000"]["Channel_000"]['complex']['real'][:]
# imag = h5_f["Measurement_000"]["Channel_000"]['complex']['imag'][:]

### Resampling

add something that checks if exists

In [35]:
# resample raw data from 165 to 80 voltage steps
h5_f["Measurement_000"]["Channel_000"]['raw_data_resample'][:] = resample(np.array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]).reshape(-1, 165), 80, axis=1)

In [31]:
# resample both real and imaginary components
h5_f["Measurement_000"]["Channel_000"]['complex']['real_resample'][:] = resample(real.reshape(num_pix, -1, num_bins), 80, axis=2)
h5_f["Measurement_000"]["Channel_000"]['complex']['imag_resample'][:] = resample(imag.reshape(num_pix, -1, num_bins), 80, axis=2)

## Visualizing Input Data


### Comparison of original and resampled data


In [1]:
# generates random pixel position and timestep
pixel = np.random.randint(0, h5_f["Measurement_000"]["Channel_000"]['complex']['real'].shape[0])
timestep = np.random.randint(0, h5_f["Measurement_000"]["Channel_000"]['complex']['real'].shape[1]) // num_bins

# plot real and imaginary components of resampled data
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# axs[0].set_title('Magnitude/Phase representation')
axs[0].plot(
    frequency_bin,
    magnitude_graph_initial_full[pixel].reshape(-1, num_bins)[timestep],
    label="amplitude",
    color="b",
)
axs[0].plot(
    wvec_freq,
    magnitude_graph_initial.reshape(num_pix, voltage_steps, -1)[pixel, timestep],
    "o",
    label="amplitude resampled",
    color="b",
)
axs[0].set(xlabel="Frequency (Hz)", ylabel="Amplitude (Arb. U.)")
ax2 = axs[0].twinx()
ax2.plot(
    frequency_bin,
    phase_graph_initial_full[pixel].reshape(-1, num_bins)[timestep],
    label="phase",
    color="r",
)
ax2.plot(
    wvec_freq,
    phase_graph_initial.reshape(num_pix, voltage_steps, -1)[pixel, timestep],
    "o",
    label="phase resampled",
    color="r",
)
ax2.set(xlabel="Frequency (Hz)", ylabel="Phase (rad)")

# axs[1].set_title('Real and Imaginary')
axs[1].plot(wvec_freq, real_resample[pixel, timestep], "s")

axs[1].plot(frequency_bin, h5_f["Measurement_000"]["Channel_000"]['real_imaginary']['real'][pixel].reshape(-1, num_bins)[timestep], label="Real")

axs[1].plot(wvec_freq, imag_resample[pixel, timestep], "s")

axs[1].plot(
    frequency_bin, h5_f["Measurement_000"]["Channel_000"]['real_imaginary']['imag'][pixel].reshape(-1, num_bins)[timestep], label="Imaginary"
)

axs[1].set(xlabel="Frequency (Hz)", ylabel="Amplitude (Arb. U.)")

fig.legend(bbox_to_anchor=(1.16, 0.93), loc="upper right", borderaxespad=0.0)
fig.tight_layout()

plt.savefig(f"{path}Assets/Figures/SHO_input.png", pad_inches=1)
plt.savefig(f"{path}Assets/Figures/SHO_input.svg", pad_inches=1)

NameError: name 'h5_f' is not defined

In [36]:
h5_f.close()